<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW2_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4%2BAI%E5%9B%9E%E9%A5%8B(%E6%9C%80%E7%B5%82%E6%95%B4%E5%90%88%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **操作說明**
  有兩種動作可供選擇



---



##   **新增/刪除資料**
新增資料

  ***日期時間自動輸入***
  1.  輸入項目（1: 早餐, 2: 午餐, 3: 晚餐, 4: 飲料, 5: 交通, 6: 就學用品, 7: 日常用品）
  2.  輸入品項
  3.  輸入金額
  4.  是否要AA分攤（yes or no）

          yes：自動除以輸入之人數並顯示實付金額在輸出區與試算表
      ---
刪除資料

  依照提供之表格輸入要刪除的資料列號





---


##   **統整資料**

Gemini的分析回饋



---



**Google Sheet**：https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing







In [2]:
# =====================================================
# 主控制程式：輸入 1 → 啟動 Gradio 記帳系統
#              輸入 2 → 執行 Gemini 收支分析
# =====================================================

choice = input("請選擇功能（1：Gradio 記帳系統，2：Gemini 收支分析）➡ ")

# =====================================================
# 🔹 選項 1：Gradio 記帳系統
# =====================================================
if choice == "1":
    import gspread
    import pandas as pd
    import datetime
    import pytz
    import gradio as gr
    from google.colab import auth
    from google.auth import default
    from google.oauth2.service_account import Credentials

    # Google Sheets 授權
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # 開啟試算表與工作表
    SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing"
    gsheets = gc.open_by_url(SPREADSHEET_URL)
    worksheet = gsheets.worksheet("工作表1")

    # ========== 功能 1：新增資料 ==========
    def add_record(category, item, amount, aa, people):
        tz = pytz.timezone("Asia/Taipei")
        now = datetime.datetime.now(tz)
        date_str = now.strftime("%Y-%m-%d")
        time_str = now.strftime("%H:%M")

        if aa == "yes" and people > 0:
            amount = amount // people

        new_data = pd.DataFrame([{
            "日期": date_str,
            "時間": time_str,
            "項目": category,
            "品項": item,
            "金額": amount
        }])

        worksheet.append_rows(values=new_data.values.tolist(), value_input_option="USER_ENTERED")
        return f"✅ 已新增資料：{category} - {item} - {amount} 元", show_records()

    # ========== 功能 2：顯示 / 刪除資料 ==========
    def show_records():
        all_data = worksheet.get_all_values()
        df = pd.DataFrame(all_data[1:], columns=all_data[0])
        df["金額"] = pd.to_numeric(df["金額"], errors="coerce")
        return df.reset_index(drop=True).reset_index().rename(columns={"index": "列號"})

    def delete_record(row_number):
        try:
            worksheet.delete_rows(int(row_number) + 2)
            return f"🗑️ 已刪除第 {row_number} 列資料", show_records()
        except Exception as e:
            return f"❌ 刪除失敗: {e}", show_records()

    # ========== Gradio 介面 ==========
    with gr.Blocks() as demo:
        gr.Markdown("# 💵 Google Sheet 記帳系統")

        records_table = gr.Dataframe(
            value=show_records(),
            label="📋 目前紀錄",
            interactive=False
        )

        with gr.Tab("新增資料"):
            category = gr.Dropdown(
                ["早餐", "午餐", "晚餐", "飲料", "交通", "就學用品", "日常用品", "其他"],
                label="項目"
            )
            item = gr.Textbox(label="品項")
            amount = gr.Number(label="金額", precision=0)
            aa = gr.Radio(["yes", "no"], label="是否AA分攤", value="no")
            people = gr.Number(label="分攤人數", value=1, precision=0, visible=False)
            add_btn = gr.Button("新增資料")
            add_output = gr.Textbox(label="結果")

            def toggle_people(aa_choice):
                if aa_choice == "yes":
                    return gr.update(visible=True, value=1)
                else:
                    return gr.update(visible=False, value=1)

            aa.change(fn=toggle_people, inputs=aa, outputs=people)

            add_btn.click(
                fn=add_record,
                inputs=[category, item, amount, aa, people],
                outputs=[add_output, records_table]
            )

        with gr.Tab("刪除資料"):
            row_number = gr.Number(label="要刪除的列號 (從 0 開始)", precision=0)
            delete_btn = gr.Button("刪除資料")
            delete_output = gr.Textbox(label="結果")

            delete_btn.click(fn=delete_record, inputs=row_number, outputs=[delete_output, records_table])

    demo.launch()

# =====================================================
# 🔹 選項 2：Gemini 收支分析
# =====================================================
elif choice == "2":
    import google.generativeai as genai
    from google.colab import userdata
    import gspread
    from google.colab import auth
    from google.auth import default
    import pandas as pd
    import datetime
    import pytz

    print("Testing Gemini 收支分析...")

    try:
        # ========== Gemini API ==========
        genai.configure(api_key=userdata.get("gemini"))
        model = genai.GenerativeModel("gemini-2.5-flash")

        # ========== Google Sheets 授權 ==========
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)

        # 用試算表 ID 連線
        SPREADSHEET_ID = "1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84"
        spreadsheet = gc.open_by_key(SPREADSHEET_ID)

        worksheet1 = spreadsheet.worksheet("工作表1")
        worksheet2 = spreadsheet.worksheet("工作表2")

        # ========== 收支分析函式 ==========
        def analyze_records():
            all_data = worksheet1.get_all_values()
            if not all_data or len(all_data) < 2:
                print("⚠️ 沒有資料可以分析")
                return

            df = pd.DataFrame(all_data[1:], columns=all_data[0])

            prompt = f"""
            以下是我的收支表：
            {df.to_string(index=False)}

            請根據資料，給我收支的總結與三點具體建議，
            包含：花費最多的項目、可能的節省方向、以及理財提醒。
            """

            response = model.generate_content(prompt, request_options={"timeout": 60})
            advice = response.text if response.text else "⚠️ Gemini 沒有回傳內容"

            tz = pytz.timezone("Asia/Taipei")
            timestamp = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

            worksheet2.append_row([timestamp, advice])

            print("✅ 已成功將分析結果寫入『工作表2』！")
            print("Gemini 建議：\n", advice)

        # 🔹 直接執行一次
        analyze_records()

    except Exception as e:
        print(f"❌ 發生錯誤: {e}")

# =====================================================
# 🔹 錯誤輸入處理
# =====================================================
else:
    print("請輸入 1 或 2。")


請選擇功能（1：Gradio 記帳系統，2：Gemini 收支分析）➡ 2
Testing Gemini 收支分析...
✅ 已成功將分析結果寫入『工作表2』！
Gemini 建議：
 根據您提供的收支表，以下是您的消費總結與具體建議：

---

### **收支總結**

在 2025-09-15 至 2025-10-01 這段期間，您的總花費為 **新台幣 3264 元**。

各項目花費分佈如下：

*   **交通:** 1240 元 (佔總花費約 37.98%)
*   **就學用品:** 1060 元 (佔總花費約 32.48%)
*   **午餐:** 330 元 (佔總花費約 10.11%)
*   **飲料:** 255 元 (佔總花費約 7.81%)
*   **晚餐:** 254 元 (佔總花費約 7.78%)
*   **日常用品:** 70 元 (佔總花費約 2.14%)
*   **早餐:** 55 元 (佔總花費約 1.68%)

---

### **三點具體建議**

#### **1. 花費最多的項目**

根據您的消費紀錄，花費最多的兩個項目是：

*   **交通 (1240 元):** 主要來自一張高達 1200 元的「定期票」。這通常是一筆固定且週期性的較大支出。此外，還有兩筆 Youbike 的費用。
*   **就學用品 (1060 元):** 包含教概課本 (800元)、工作服與護目鏡 (235元) 和筆 (25元)。這些可能是開學初期或特定課程所需的較大一次性或低頻率支出。

#### **2. 可能的節省方向**

*   **重新評估交通方式：**
    *   **定期票效益：** 定期票是一筆較大的預付費用。如果您的使用頻率高且通勤距離遠，這筆費用可能已經是最划算的選擇。但如果您發現有些日子或期間使用頻率不高，可以評估是否改為儲值卡或單次購票會更省錢。
    *   **Youbike使用：** 雖然單次費用不高，但如果能與定期票涵蓋的交通方式結合，或以步行取代短程的Youbike，積少成多也能省下一些費用。
*   **控制餐飲與飲料支出：**
    *   **飲料：** 飲料合計支出 255 元，平均每天約 15-20 元，雖然單筆金額不大，但累積起來是僅次於交通和就學用品的第三大開銷。考慮自